# Gallery Example: D/M/1 Queue (Deterministic Arrivals)

This example demonstrates a D/M/1 queueing system:
- **Arrivals**: Deterministic inter-arrival times (constant)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

Deterministic arrivals represent perfectly regular arrival patterns with zero variability (C² = 0).

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_detm1(): Network {    """Create D/M/1 queueing model"""    model = Network("D/M/1")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.FCFS)    sink = Sink(model, "mySink")        # Block 2: classes    oclass = OpenClass(model, "myClass")    # Deterministic arrivals with inter-arrival time = 1 (rate = 1)    source.setArrival(oclass, Det(1.0))         # Exponential service with rate μ=2    queue.setService(oclass, Exp(2))        # Block 3: topology    P = model.initRoutingMatrix()    P.addRoute(oclass, source, queue, 1.0)    P.addRoute(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_detm1()println(f"Arrival pattern: Deterministic (λ = 1, C²ₐ = 0)")println(f"Service pattern: Exponential (μ = 2, C²ₛ = 1)")println(f"Utilization: ρ = λ/μ = 1/2 = 0.5")

## Theoretical Analysis for D/M/1

For D/M/1 with:
- **Arrival rate**: λ = 1 (deterministic inter-arrival time = 1)
- **Service rate**: μ = 2 (exponential)
- **Utilization**: ρ = λ/μ = 0.5

Deterministic arrivals characteristics:
- **Inter-arrival time**: Constant = 1.0
- **Variance**: 0 (no variability)
- **Coefficient of Variation**: C²ₐ = 0

D/M/1 typically performs much better than M/M/1 due to zero arrival variability.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare D/M/1 with M/M/1 - effect of arrival variability
println("\n=== Impact of Arrival Variability: D/M/1 vs M/M/1 ===")

fun create_equivalent_mm1(): Network {
    """Create equivalent M/M/1 model"""
    val model_mm1 = Network("M/M/1-Equivalent")
    val source = Source(model_mm1, "Source")
    val queue = Queue(model_mm1, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_mm1, "Sink")
    
    val oclass = OpenClass(model_mm1, "Class")
    source.setArrival(oclass, Exp(1))  # Same mean arrival rate
    queue.setService(oclass, Exp(2))   # Same service rate
    
    val P = model_mm1.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

val model_mm1 = create_equivalent_mm1()
val solver_dm1 = MVA(model)
val solver_mm1 = MVA(model_mm1)

val avg_table_dm1 = solver_dm1.avgTable
val avg_table_mm1 = solver_mm1.avgTable

println("D/M/1 Results (deterministic arrivals):")
println(avg_table_dm1)

println("\nM/M/1 Results (exponential arrivals):")
println(avg_table_mm1)
// Extract performance metrics for comparison
val dm1_resp = float(avg_table_dm1.iloc[1, 2])  # D/M/1 response time
val dm1_length = float(avg_table_dm1.iloc[1, 3])  # D/M/1 queue length

val mm1_resp = float(avg_table_mm1.iloc[1, 2])  # M/M/1 response time
val mm1_length = float(avg_table_mm1.iloc[1, 3])  # M/M/1 queue length

println(f"\nVariability Impact Comparison:")
println(f"Arrival Distribution | C²ₐ | Response Time | Queue Length | Performance")
println("-" * 70)
println(f"Deterministic        |  0  |     {dm1_resp:.4f}    |    {dm1_length:.4f}     | Best")
println(f"Exponential          |  1  |     {mm1_resp:.4f}    |    {mm1_length:.4f}     | Baseline")

val improvement = (mm1_resp - dm1_resp) / mm1_resp * 100 if mm1_resp > dm1_resp else 0
println(f"\nDeterministic Advantage: {improvement:.1f}% better response time vs exponential arrivals")
println(f"Queue length reduction: {(mm1_length - dm1_length) / mm1_length * 100:.1f}%")
println("\nKey insight: Eliminating arrival variability significantly improves performance.")

In [ ]:
// Compare with M/D/1 (deterministic service)
println("\n=== D/M/1 vs M/D/1: Arrival vs Service Variability ===")

fun create_md1_model(): Network {
    """Create M/D/1 model (exponential arrivals, deterministic service)"""
    val model_md1 = Network("M/D/1")
    val source = Source(model_md1, "Source")
    val queue = Queue(model_md1, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_md1, "Sink")
    
    val oclass = OpenClass(model_md1, "Class")
    source.setArrival(oclass, Exp(1))      # Exponential arrivals
    queue.setService(oclass, Det(0.5))     # Deterministic service time = 0.5
    
    val P = model_md1.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_md1.link(P)
    
    return model_md1

fun create_dd1_model(): Network {
    """Create D/D/1 model (both deterministic)"""
    val model_dd1 = Network("D/D/1")
    val source = Source(model_dd1, "Source")
    val queue = Queue(model_dd1, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_dd1, "Sink")
    
    val oclass = OpenClass(model_dd1, "Class")
    source.setArrival(oclass, Det(1.0))    # Deterministic arrivals
    queue.setService(oclass, Det(0.5))     # Deterministic service
    
    val P = model_dd1.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_dd1.link(P)
    
    return model_dd1

try:
    val model_md1 = create_md1_model()
    val model_dd1 = create_dd1_model()
    
    val solver_md1 = MVA(model_md1)
    val solver_dd1 = MVA(model_dd1)
    
    val avg_table_md1 = solver_md1.avgTable
    val avg_table_dd1 = solver_dd1.avgTable
    
    val md1_resp = float(avg_table_md1.iloc[1, 2])
    val dd1_resp = float(avg_table_dd1.iloc[1, 2])
    
    println("Variability Impact Matrix:")
    println("")
    println("System    | Arrivals C²ₐ | Service C²ₛ | Response Time | Variability Effect")
    println("-" * 75)
    println(f"D/D/1     |      0       |      0      |     {dd1_resp:.4f}    | No variability (best)")
    println(f"D/M/1     |      0       |      1      |     {dm1_resp:.4f}    | Service variability only")
    println(f"M/D/1     |      1       |      0      |     {md1_resp:.4f}    | Arrival variability only")
    println(f"M/M/1     |      1       |      1      |     {mm1_resp:.4f}    | Both variabilities")
    
    println(f"\nVariability Impact Ranking (best to worst):")
    val results = [(dd1_resp, "D/D/1"), (dm1_resp, "D/M/1"), (md1_resp, "M/D/1"), (mm1_resp, "M/M/1")]
    results.sort()
    
    for i, (resp_time, system) in enumerate(results, 1):
        println(f"{i}. {system}: {resp_time:.4f}")
    
    println(f"\nKey Insights:")
    println(f"• Arrival variability impact: {(md1_resp - dd1_resp) / dd1_resp * 100:.1f}% increase")
    println(f"• Service variability impact: {(dm1_resp - dd1_resp) / dd1_resp * 100:.1f}% increase")
    println(f"• Combined effect: {(mm1_resp - dd1_resp) / dd1_resp * 100:.1f}% increase")
    println(f"• Variability in {"arrivals" if md1_resp > dm1_resp else "service"} has greater impact")

except Exception as e:
    println(f"Detailed comparison error: {e}")
    println("Note: Some solvers may not support deterministic distributions in all configurations.")
    println("The key principle remains: reducing variability improves queueing performance.")